# Sunbird RC API - Water Facility Demo

This notebook demonstrates how to interact with Sunbird RC Registry API for Water Facilities.

## Prerequisites
- Sunbird RC services running (use `./start-sunbird.sh`)
- Registry API available at http://localhost:8081
- WaterFacility schema configured

## Setup

In [1]:
import requests
import json
import pandas as pd
from datetime import datetime
from IPython.display import display, Markdown, JSON

# Configuration
BASE_URL = "http://localhost:8081/api/v1"
HEADERS = {"Content-Type": "application/json"}

print("✅ Setup complete!")
print(f"Registry API: {BASE_URL}")

✅ Setup complete!
Registry API: http://localhost:8081/api/v1


## 1. Check Registry Health

In [2]:
response = requests.get("http://localhost:8081/health")
health = response.json()

print(f"Status: {health['result']['healthy']}")
print(f"Service: {health['result']['name']}")
display(JSON(health, expanded=True))

Status: True
Service: sunbirdrc-registry-api


<IPython.core.display.JSON object>

## 2. List All Water Facilities

In [3]:
response = requests.get(f"{BASE_URL}/WaterFacility", headers=HEADERS)
data = response.json()

print(f"Total Water Facilities: {data['totalCount']}\n")

if data['totalCount'] > 0:
    df = pd.DataFrame(data['data'])
    # Select key columns (only include columns that exist)
    desired_columns = ['geoCode', 'wfId', 'waterPointType', 'extractionType', 'pumpType', 'owner', 'osid']
    columns = [col for col in desired_columns if col in df.columns]
    display(df[columns])
    
    # Show location details separately
    if 'location' in df.columns:
        print("\nLocation Details:")
        location_df = pd.json_normalize(df['location'])
        location_df['geoCode'] = df['geoCode'].values
        display(location_df[['geoCode', 'county', 'district', 'community']])
else:
    print("No water facilities found")

Total Water Facilities: 0

No water facilities found


## 3. Create a New Water Facility

In [4]:
def test_new_facility():
    new_facility = {
        "geoCode": "test001",
        "waterPointType": "Protected dug well",
        "location": {
            "county": "Bong",
            "district": "Kpaai",
            "community": "Test Community",
            "coordinates": {
                "lat": 6.99428667,
                "lon": -9.46091172,
                "elevation": 286.4
            }
        },
        "extractionType": "Manual",
        "pumpType": "Afridev",
        "hasDepthInfo": True,
        "depthMetres": 15.0,
        "installer": "NGO",
        "owner": "Community",
        "funder": "USAID",
        "photoUrl": "https://example.com/photo.jpg"
    }
    
    response = requests.post(
        f"{BASE_URL}/WaterFacility",
        headers=HEADERS,
        json=new_facility
    )
    
    if response.status_code == 200:
        result = response.json()
        osid = result['result']['WaterFacility']['osid']
        print("Water Facility created successfully!")
        print(f"osid: {osid}")
        
        # Fetch the created facility to show the generated wfId
        fetch_response = requests.get(f"{BASE_URL}/WaterFacility/{osid}")
        if fetch_response.status_code == 200:
            created_facility = fetch_response.json()
            print(f"wfId: {created_facility.get('wfId', 'N/A')}")
            print(f"\nGenerated wfId format: WF-<COUNTY>-<DISTRICT>-<TYPE>-<HASH>")
            display(JSON(created_facility, expanded=True))
    else:
        print(f"Error: {response.status_code}")
        print(response.text)

In [5]:
test_new_facility()

Water Facility created successfully!
osid: 1-3d2d6903-54b4-49c1-bbff-489f620bdb06
wfId: WF-BON-KPA-PDW-6FA851

Generated wfId format: WF-<COUNTY>-<DISTRICT>-<TYPE>-<HASH>


<IPython.core.display.JSON object>

## 4. Get a Specific Water Facility by ID

In [6]:
# Get the first facility's ID
response = requests.get(f"{BASE_URL}/WaterFacility")
facilities = response.json()['data']

if facilities:
    facility_id = facilities[0]['osid']
    print(f"Fetching water facility: {facility_id}\n")
    response = requests.get(f"{BASE_URL}/WaterFacility/{facility_id}")
    facility = response.json()
    display(JSON(facility, expanded=True))
else:
    print("No facilities found")

Fetching water facility: 1-3d2d6903-54b4-49c1-bbff-489f620bdb06



<IPython.core.display.JSON object>

## 5. Update a Water Facility

In [7]:
# Get first facility
response = requests.get(f"{BASE_URL}/WaterFacility")
facilities = response.json()['data']

if facilities:
    facility_id = facilities[0]['osid']
    
    # Update the facility
    update_data = facilities[0].copy()
    update_data['depthMetres'] = 25.0  # Update depth
    update_data['funder'] = 'Updated Funder Organization'  # Update funder
    
    response = requests.put(
        f"{BASE_URL}/WaterFacility/{facility_id}",
        headers=HEADERS,
        json=update_data
    )
    
    if response.status_code == 200:
        print("Water Facility updated successfully!")
        display(JSON(response.json(), expanded=True))
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
else:
    print("No facilities available to update")

Water Facility updated successfully!


<IPython.core.display.JSON object>

## 6. Search Water Facilities

In [8]:
search_query = {
    "filters": {
        "waterPointType": {"eq": "Protected dug well"}
    }
}

response = requests.post(
    f"{BASE_URL}/WaterFacility/search",
    headers=HEADERS,
    json=search_query
)

if response.status_code == 200:
    results = response.json()
    print(f"Found {results['totalCount']} Protected dug wells\n")

    if results['totalCount'] > 0:
        df = pd.DataFrame(results['data'])
        # Include wfId in the display
        desired_columns = ['wfId', 'geoCode', 'waterPointType', 'extractionType', 'pumpType', 'owner']
        columns = [col for col in desired_columns if col in df.columns]
        display(df[columns])

        # Show location details
        if 'location' in df.columns:
            print("\nLocation Details:")
            location_df = pd.json_normalize(df['location'])
            location_df['geoCode'] = df['geoCode'].values
            if 'wfId' in df.columns:
                location_df['wfId'] = df['wfId'].values
                display(location_df[['wfId', 'geoCode', 'county', 'district', 'community']])
            else:
                display(location_df[['geoCode', 'county', 'district', 'community']])
    else:
        print("No results found")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

Found 1 Protected dug wells



,wfId,geoCode,waterPointType,extractionType,pumpType,owner
0,WF-BON-KPA-PDW-6FA851,test001,Protected dug well,Manual,Afridev,Community



Location Details:


,wfId,geoCode,county,district,community
0,WF-BON-KPA-PDW-6FA851,test001,Bong,Kpaai,Test Community


## 7. Bulk Create Water Facilities

In [9]:
# Sample bulk data with Liberia water points
bulk_facilities = [
    {
        "geoCode": "w4q6yvl",
        "waterPointType": "Tube well or borehole",
        "location": {
            "county": "Nimba",
            "district": "Buu-Yao",
            "community": "Say's",
            "coordinates": {"lat": 6.99428667, "lon": -9.46091172, "elevation": 286.4}
        },
        "extractionType": "Manual",
        "pumpType": "India Mark",
        "hasDepthInfo": False,
        "installer": "Government",
        "owner": "Community",
        "funder": "LWC",
        "photoUrl": "https://flowliberia.s3.amazonaws.com/images/cfd2e566-18ee-4952-b08b-31db5fef24bf.jpg"
    },
    {
        "geoCode": "w4rdu39",
        "waterPointType": "Tube well or borehole",
        "location": {
            "county": "Bong",
            "district": "Kpaai",
            "community": "Siawolor Town",
            "coordinates": {"lat": 6.99448282, "lon": -9.46136413, "elevation": 290.7}
        },
        "extractionType": "Electrical",
        "pumpType": "Kardia",
        "hasDepthInfo": True,
        "depthMetres": 108.0,
        "installer": "NGO",
        "owner": "Health Facility",
        "funder": "USAID",
        "photoUrl": "https://flowliberia.s3.amazonaws.com/images/845285a9-6a39-44a7-a259-9327db56a5a0.jpg"
    },
    {
        "geoCode": "w4qdecx",
        "waterPointType": "Protected dug well",
        "location": {
            "county": "Lofa",
            "district": "Zorzor",
            "community": "Zoe",
            "coordinates": {"lat": 6.99431151, "lon": -9.46096315, "elevation": 279.7}
        },
        "extractionType": "Manual",
        "pumpType": "Afridev",
        "hasDepthInfo": False,
        "installer": "NGO",
        "owner": "Health Facility",
        "funder": "LWC",
        "photoUrl": "https://flowliberia.s3.amazonaws.com/images/591f7c89-7dc3-44af-a684-e4ab8e82a0c6.jpg"
    },
    {
        "geoCode": "rugw4rt",
        "waterPointType": "Rainwater (harvesting)",
        "location": {
            "county": "Grand Gedeh",
            "district": "Tchien",
            "community": "Zwedru",
            "coordinates": {"lat": 6.061545, "lon": -8.13854, "elevation": 254.8}
        },
        "numTaps": 1.0,
        "hasDepthInfo": False,
        "installer": "Government",
        "owner": "Community",
        "funder": "Unicef",
        "photoUrl": "https://flowliberia.s3.amazonaws.com/images/668b7ad7-4561-4850-90bd-8c7ff7dab977.jpg"
    },
    {
        "geoCode": "w4t1uky",
        "waterPointType": "Protected spring",
        "location": {
            "county": "Bong",
            "district": "Boinsen",
            "community": "Samai",
            "coordinates": {"lat": 6.99476405, "lon": -9.46100939, "elevation": 293.1}
        },
        "installer": "Private",
        "owner": "Private Individual",
        "funder": "Private",
        "photoUrl": "https://flowliberia.s3.amazonaws.com/images/dbd8c589-b69a-4794-a14b-48672812920a.jpg"
    }
]

print(f"Creating {len(bulk_facilities)} water facilities...\n")

results = []
for i, facility in enumerate(bulk_facilities, 1):
    response = requests.post(
        f"{BASE_URL}/WaterFacility",
        headers=HEADERS,
        json=facility
    )

    if response.status_code == 200:
        result = response.json()
        osid = result['result']['WaterFacility']['osid']
        
        # Fetch the created facility to get the generated wfId
        fetch_response = requests.get(f"{BASE_URL}/WaterFacility/{osid}")
        wfId = "N/A"
        if fetch_response.status_code == 200:
            created = fetch_response.json()
            wfId = created.get('wfId', 'N/A')
        
        print(f"{i}. {facility['geoCode']} ({facility['waterPointType']})")
        print(f"   wfId: {wfId}")
        print(f"   osid: {osid}\n")
        results.append({'status': 'success', 'osid': osid, 'wfId': wfId, 'geoCode': facility['geoCode']})
    else:
        print(f"{i}. {facility['geoCode']} - FAILED ({response.status_code})")
        print(f"   Error: {response.text}\n")
        results.append({'status': 'failed', 'geoCode': facility['geoCode'], 'error': response.text})

print(f"{'='*50}")
print(f"Created {len([r for r in results if r['status'] == 'success'])} facilities")
print(f"Failed {len([r for r in results if r['status'] == 'failed'])} facilities")

Creating 5 water facilities...

1. w4q6yvl (Tube well or borehole)
   wfId: WF-NIM-BUU-TWB-1E5B56
   osid: 1-6ae82c57-758a-4e97-bfa5-32159a30024a

2. w4rdu39 (Tube well or borehole)
   wfId: WF-BON-KPA-TWB-6F1F9D
   osid: 1-23b39922-195e-4733-b06c-56bc62d18fd0

3. w4qdecx (Protected dug well)
   wfId: WF-LOF-ZOR-PDW-E359B1
   osid: 1-adf1462c-f1e5-45ba-ae44-8bf231dd90ae

4. rugw4rt (Rainwater (harvesting))
   wfId: WF-GRA-TCH-RWH-D1DB1C
   osid: 1-c80cc49a-edc2-453f-80aa-cb3d25705ec9

5. w4t1uky (Protected spring)
   wfId: WF-BON-BOI-PS-168EAB
   osid: 1-fb7fdabd-3fa8-4ffa-9bea-5ba470ff4d1d

Created 5 facilities
Failed 0 facilities


## 7.1 Test Duplicate Rejection (wfId uniqueness)

The wfId is generated based on a hash of facility attributes (geoCode, waterPointType, location).
Attempting to create a facility with the same attributes should be rejected due to the unique constraint on wfId.

In [10]:
# Test: Attempt to create a duplicate facility
# This should fail because wfId is unique based on facility attributes

duplicate_facility = {
    "geoCode": "w4q6yvl",  # Same geoCode as bulk create
    "waterPointType": "Tube well or borehole",  # Same type
    "location": {
        "county": "Nimba",  # Same location
        "district": "Buu-Yao",
        "community": "Say's",
        "coordinates": {"lat": 6.99428667, "lon": -9.46091172, "elevation": 286.4}
    },
    "extractionType": "Electrical",  # Different extraction - should not matter
    "pumpType": "Afridev",  # Different pump - should not matter
    "installer": "Private",  # Different installer - should not matter
    "owner": "Private Individual",  # Different owner - should not matter
    "funder": "Different Funder"  # Different funder - should not matter
}

print("Testing duplicate rejection...")
print(f"Attempting to create: {duplicate_facility['geoCode']}")
print(f"Location: {duplicate_facility['location']['county']}, {duplicate_facility['location']['district']}")
print()

response = requests.post(
    f"{BASE_URL}/WaterFacility",
    headers=HEADERS,
    json=duplicate_facility
)

if response.status_code == 200:
    result = response.json()
    osid = result['result']['WaterFacility']['osid']
    print(f"Unexpected: Facility was created with osid: {osid}")
    
    # Fetch to see the wfId
    fetch_response = requests.get(f"{BASE_URL}/WaterFacility/{osid}")
    if fetch_response.status_code == 200:
        created = fetch_response.json()
        print(f"wfId: {created.get('wfId', 'N/A')}")
else:
    print(f"Expected behavior: Duplicate rejected!")
    print(f"Status code: {response.status_code}")
    try:
        error_detail = response.json()
        print(f"Error: {json.dumps(error_detail, indent=2)}")
    except:
        print(f"Error: {response.text}")

Testing duplicate rejection...
Attempting to create: w4q6yvl
Location: Nimba, Buu-Yao

Expected behavior: Duplicate rejected!
Status code: 500
Error: {
  "id": "sunbird-rc.registry.post",
  "ver": "1.0",
  "ets": 1771310648878,
  "params": {
    "resmsgid": "",
    "msgid": "f9429f35-a4ed-42c2-9444-9cd863ac7ff1",
    "err": "",
    "status": "UNSUCCESSFUL",
    "errmsg": "dev.sunbirdrc.registry.exception.UniqueIdentifierException: dev.sunbirdrc.registry.exception.UniqueIdentifierException$GenerateException: Unable to generate id: Duplicate WaterFacility: A water point with wfId 'WF-NIM-BUU-TWB-1E5B56' already exists. Water points with the same geoCode, type, and location are not allowed."
  },
  "responseCode": "OK",
  "result": {}
}


## 8. Export to CSV

In [11]:
response = requests.get(f"{BASE_URL}/WaterFacility")
data = response.json()

if data['totalCount'] > 0:
    df = pd.DataFrame(data['data'])
    
    # Flatten location object for export
    if 'location' in df.columns:
        location_df = pd.json_normalize(df['location'])
        location_df.columns = [f'location_{col}' for col in location_df.columns]
        df = pd.concat([df.drop('location', axis=1), location_df], axis=1)
    
    # Select columns to export (updated for new schema)
    export_columns = [
        'geoCode', 'wfId', 'waterPointType', 'extractionType', 'pumpType',
        'location_county', 'location_district', 'location_community',
        'location_coordinates.lat', 'location_coordinates.lon', 'location_coordinates.elevation',
        'numTaps', 'hasDepthInfo', 'depthMetres',
        'installer', 'owner', 'funder', 'photoUrl',
        'osid', 'osCreatedAt'
    ]
    
    # Only include columns that exist in the dataframe
    available_columns = [col for col in export_columns if col in df.columns]
    
    filename = f'water_facilities_export.csv'
    df[available_columns].to_csv(filename, index=False)
    
    print(f"Exported {len(df)} water facilities to: {filename}")
    display(df[available_columns].head())
else:
    print("No water facilities to export")

Exported 6 water facilities to: water_facilities_export.csv


,geoCode,wfId,waterPointType,extractionType,pumpType,location_county,location_district,location_community,location_coordinates.lat,location_coordinates.lon,location_coordinates.elevation,numTaps,hasDepthInfo,depthMetres,installer,owner,funder,photoUrl,osid,osCreatedAt
0,test001,WF-BON-KPA-PDW-6FA851,Protected dug well,Manual,Afridev,Bong,Kpaai,Test Community,6.994287,-9.460912,286.4,NaN,True,25.0,NGO,Community,Updated Funder Organization,https://example.com/photo.jpg,1-3d2d6903-54b4-49c1-bbff-489f620bdb06,2026-02-17T06:44:08.682Z
1,w4q6yvl,WF-NIM-BUU-TWB-1E5B56,Tube well or borehole,Manual,India Mark,Nimba,Buu-Yao,Say's,6.994287,-9.460912,286.4,NaN,False,NaN,Government,Community,LWC,https://flowliberia.s3.amazonaws.com/images/cf...,1-6ae82c57-758a-4e97-bfa5-32159a30024a,2026-02-17T06:44:08.789Z
2,w4rdu39,WF-BON-KPA-TWB-6F1F9D,Tube well or borehole,Electrical,Kardia,Bong,Kpaai,Siawolor Town,6.994483,-9.461364,290.7,NaN,True,108.0,NGO,Health Facility,USAID,https://flowliberia.s3.amazonaws.com/images/84...,1-23b39922-195e-4733-b06c-56bc62d18fd0,2026-02-17T06:44:08.809Z
3,w4qdecx,WF-LOF-ZOR-PDW-E359B1,Protected dug well,Manual,Afridev,Lofa,Zorzor,Zoe,6.994312,-9.460963,279.7,NaN,False,NaN,NGO,Health Facility,LWC,https://flowliberia.s3.amazonaws.com/images/59...,1-adf1462c-f1e5-45ba-ae44-8bf231dd90ae,2026-02-17T06:44:08.826Z
4,rugw4rt,WF-GRA-TCH-RWH-D1DB1C,Rainwater (harvesting),NaN,NaN,Grand Gedeh,Tchien,Zwedru,6.061545,-8.138540,254.8,1.0,False,NaN,Government,Community,Unicef,https://flowliberia.s3.amazonaws.com/images/66...,1-c80cc49a-edc2-453f-80aa-cb3d25705ec9,2026-02-17T06:44:08.842Z


## 9. Delete All Water Facilities

In [12]:
# Delete all Water Facilities
def delete_all_facilities():
    response = requests.get(f"{BASE_URL}/WaterFacility")
    facilities = response.json()['data']
    
    if facilities:
        print(f"Deleting {len(facilities)} water facilities...\n")
        
        deleted = 0
        failed = 0
        for facility in facilities:
            osid = facility['osid']
            geoCode = facility.get('geoCode', 'Unknown')
            
            response = requests.delete(f"{BASE_URL}/WaterFacility/{osid}")
            
            if response.status_code == 200:
                print(f"Deleted: {geoCode} ({osid})")
                deleted += 1
            else:
                print(f"Failed to delete: {geoCode} ({osid})")
                failed += 1
        
        print(f"\nDeleted {deleted} facilities")
        if failed > 0:
            print(f"Failed {failed} facilities")
    else:
        print("No water facilities to delete")

In [13]:
delete_all_facilities()

Deleting 6 water facilities...

Deleted: test001 (1-3d2d6903-54b4-49c1-bbff-489f620bdb06)
Deleted: w4q6yvl (1-6ae82c57-758a-4e97-bfa5-32159a30024a)
Deleted: w4rdu39 (1-23b39922-195e-4733-b06c-56bc62d18fd0)
Deleted: w4qdecx (1-adf1462c-f1e5-45ba-ae44-8bf231dd90ae)
Deleted: rugw4rt (1-c80cc49a-edc2-453f-80aa-cb3d25705ec9)
Deleted: w4t1uky (1-fb7fdabd-3fa8-4ffa-9bea-5ba470ff4d1d)

Deleted 6 facilities


## 10. Make Sure That Hard Delete is Enabled

In [14]:
test_new_facility()

Water Facility created successfully!
osid: 1-9ac41b1c-b212-4c92-819b-54f5b1ed1e81
wfId: WF-BON-KPA-PDW-6FA851

Generated wfId format: WF-<COUNTY>-<DISTRICT>-<TYPE>-<HASH>


<IPython.core.display.JSON object>

In [15]:
delete_all_facilities()

Deleting 1 water facilities...

Deleted: test001 (1-9ac41b1c-b212-4c92-819b-54f5b1ed1e81)

Deleted 1 facilities


## Summary

This notebook demonstrated:
- Checking registry health
- Listing all water facilities
- Creating new water facilities with auto-generated wfId
- Reading specific facilities
- Updating facilities
- Searching facilities by type
- Bulk operations with wfId display
- Testing duplicate rejection (wfId uniqueness)
- Exporting to CSV
- Deleting all facilities

### wfId Generation

The `wfId` is automatically generated when creating a WaterFacility using the format:
```
WF-<COUNTY_ABBR>-<DISTRICT_ABBR>-<TYPE_CODE>-<HASH>
```

**Example:** `WF-NIM-BUU-TWB-CB37DF`

**Components:**
- `WF-` - Fixed prefix for Water Facility
- `COUNTY_ABBR` - First 3 alphanumeric characters of county (uppercase)
- `DISTRICT_ABBR` - First 3 alphanumeric characters of district (uppercase)
- `TYPE_CODE` - Water point type code:
  - `PDW` - Protected dug well
  - `UDW` - Unprotected dug well
  - `TWB` - Tube well or borehole
  - `PS` - Protected spring
  - `US` - Unprotected spring
  - `PWD` - Piped water into dwelling/plot/yard
  - `PTS` - Public tap/standpipe
  - `UEB` - Unequipped borehole
  - `RWH` - Rainwater (harvesting)
  - `SSD` - Sand/Sub-surface dam (with well or standpipe)
  - `OTH` - Other
- `HASH` - 6-character SHA-256 hash of normalized attributes

**Hash inputs (normalized to lowercase, trimmed):**
- geoCode
- waterPointType
- location.county
- location.district
- location.community

**Uniqueness:** The wfId is marked as a unique index field. Duplicate facilities (same core attributes) will be rejected.

### Water Facility Schema Fields

**Required Fields:**
- `geoCode` - Unique geographic code identifier
- `location` - Administrative location object
  - `county` - County name
  - `district` - District name
  - `community` - Community name
  - `coordinates` - Geographic coordinates (optional)
    - `lat` - Latitude
    - `lon` - Longitude
    - `elevation` - Elevation in meters
- `waterPointType` - Type of water point (enum)

**Auto-generated Fields:**
- `wfId` - System-generated unique ID (format: WF-XXX-XXX-TYPE-HASH)

**Optional Fields:**
- `extractionType` - Water extraction method (Manual, Electrical, Solar, Other)
- `extractionTypeOther` - Other extraction type specification
- `pumpType` - Type of pump installed (Afridev, Consallen, India Mark, Kardia, Rope pump, Vergnet, Other)
- `pumpTypeOther` - Other pump type specification
- `numTaps` - Number of taps
- `hasDepthInfo` - Whether depth information is available
- `depthMetres` - Depth in metres
- `installer` - Entity that installed (Government, NGO, Private, Other)
- `installerOther` - Other installer specification
- `owner` - Owner of the water point (Community, Private Individual, School, NGO, Health Facility, Other institution, CBO, Private, Unknown, Other)
- `funder` - Funding organization or entity
- `photoUrl` - URL to photo of the water point

### Water Point Types Supported
- Protected dug well (PDW)
- Unprotected dug well (UDW)
- Tube well or borehole (TWB)
- Protected spring (PS)
- Unprotected spring (US)
- Piped water into dwelling/plot/yard (PWD)
- Public tap/standpipe (PTS)
- Unequipped borehole (UEB)
- Rainwater (harvesting) (RWH)
- Sand/Sub-surface dam (with well or standpipe) (SSD)
- Other (OTH)

For more information, see the WaterFacility schema at `java/registry/src/main/resources/public/_schemas/WaterFacility.json`